In [1]:
import sys
sys.path.append('../')

from clients import MysqlClient
from dotenv import load_dotenv
import os

load_dotenv()
os.getenv('AWS_ACCESS_KEY_ID')
os.getenv('AWS_SECRET_ACCESS_KEY')
mysql = MysqlClient()

In [74]:
mysql.get_table_names()

Tables : 
daum_movies
daum_pop_movies_update_info
daum_ratings
movies
ratings


In [18]:
sql = """
SELECT *
FROM daum_movies
WHERE movieId = '1'
"""

sql = """
SHOW TABLES
"""

sql = """
CREATE TABLE daum_pop_movies AS
SELECT * FROM daum_movies WHERE 1=0;
"""
with mysql.get_connection().cursor() as cursor:
    cursor.execute(sql)
    data = cursor.fetchall()

In [16]:
cursor.fetchall()

()

In [17]:
data

(('daum_movies',), ('daum_ratings',), ('movies',), ('ratings',))

In [6]:
cursor.fetchall()

()

In [19]:
mysql.get_table_names()

Tables : 
daum_movies
daum_pop_movies
daum_ratings
movies
ratings


In [23]:
import pandas as pd

sql = """
select *
from daum_pop_movies
"""

with mysql.get_connection() as connection:
    df = pd.read_sql(sql=sql, con=connection)

/var/folders/cg/2vt5wbq5767fsxv0005k5x4m0000gn/T/ipykernel_62885/2521560428.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sql, con=connection)


In [25]:
df

,movieId,titleKo,titleEn,synopsis,cast,mainPageUrl,posterUrl,numOfSiteRatings


In [89]:
host = "pseudorec.cvhv2t0obyv3.ap-northeast-2.rds.amazonaws.com"
db_url = f"mysql+mysqlconnector://admin:{os.getenv('RDS_MYSQL_PW')}@{host}:3306/movielens25m"

In [90]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(db_url)

In [91]:
grouped_df = daum_ratings.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
rating_calculated_df = grouped_df.sort_values('mean', ascending=False)
rating_calculated_df

rating_calculated_df.to_sql(name='daum_pop_movies', con=engine, if_exists='replace', index=False)

30157

In [92]:
sql = """
select *
from daum_pop_movies
"""
df = pd.read_sql(sql=sql, con=engine)

In [93]:
df

,movieId,mean,count
0,33335,10.0,4
1,30103,10.0,1
2,30182,10.0,1
3,106973,10.0,1
4,106974,10.0,1
...,...,...,...
30152,101049,0.0,1
30153,44097,0.0,1
30154,101210,0.0,1
30155,101223,0.0,1


In [ ]:
만들었으니 계산식 .py에 만들고 
시간 저장하는 테이블 또 만들고

.py 파일 작성하자


In [83]:
create_table_query = """
CREATE TABLE daum_pop_movies (
                                movieId INT PRIMARY KEY,
                                mean FLOAT,
                                count INT
                                )
"""


with mysql.get_connection().cursor() as cursor:
    cursor.execute(create_table_query)

In [84]:
mysql.get_table_names()

Tables : 
daum_movies
daum_pop_movies
daum_ratings
movies
ratings


In [94]:
import pandas as pd
sql = """
select *
from daum_pop_movies
"""
df = pd.read_sql(sql=sql, con=engine)

df

,movieId,mean,count
0,33335,10.0,4
1,30103,10.0,1
2,30182,10.0,1
3,106973,10.0,1
4,106974,10.0,1
...,...,...,...
30152,101049,0.0,1
30153,44097,0.0,1
30154,101210,0.0,1
30155,101223,0.0,1


In [96]:
df[df['count'] > 500].sort_values('mean', ascending=False).head(100)

,movieId,mean,count
3620,76325,9.563291,2686
4035,91662,9.462662,616
4036,108595,9.455903,1406
4045,108944,9.452693,687
4143,119688,9.395866,629
...,...,...,...
25155,120834,4.832727,550
25325,119835,4.667732,626
25406,125372,4.658672,1084
25494,94647,4.569767,688


In [20]:
from datetime import datetime

current_timestamp = int(datetime.now().timestamp())
print(current_timestamp)


1705737470


In [14]:
sql

'\nselect *\nfrom daum_pop_movies_update_info\n'

In [ ]:
ALTER TABLE daum_pop_movies_update_info
MODIFY COLUMN last_update_timestamp INT NOT NULL;

In [30]:
import time
current_timestamp = int(time.time())
current_timestamp

1705737843

In [26]:
import time
current_timestamp = int(time.time())

sql = "INSERT INTO daum_pop_movies_update_info (last_update_timestamp) VALUES (%s);"
sql


with mysql.get_connection() as connection:
    cursor = connection.cursor()
    # cursor.execute(sql)
    cursor.execute(sql, (current_timestamp,))
    
    connection.commit()

In [56]:
current_timestamp

1705736847

# 삭제 (평점 테이블 삭제하지않게 조심)

In [75]:
sql = """
DROP TABLE daum_pop_movies_update_info
"""

with mysql.get_connection() as connection:
    cursor = connection.cursor()
    cursor.execute(sql)
    connection.commit()

In [76]:
mysql.get_table_names()

Tables : 
daum_movies
daum_ratings
movies
ratings


In [32]:
from collections import Counter

In [43]:
daum_ratings = mysql.get_daum_ratings()
daum_ratings = daum_ratings[daum_ratings['nickName'].map(lambda x: x not in ['휴면 사용자', '', '닉네임을 등록해 주세요', '닉네임'])]
daum_movies = mysql.get_daum_movies()


/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/notebooks/../clients.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql='select * from daum_ratings', con=connection)
/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/notebooks/../clients.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql='select * from daum_movies', con=connection)


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# MySQL 연결 문자열
host = "pseudorec.cvhv2t0obyv3.ap-northeast-2.rds.amazonaws.com"
db_url = f"mysql+mysqlconnector://admin:{os.getenv('RDS_MYSQL_PW')}@{host}:3306/movielens25m"


# MySQL 테이블에 저장
engine = create_engine(db_url)
your_dataframe.to_sql(name='your_table_name', con=engine, if_exists='replace', index=False)


In [44]:
daum_movies

,movieId,titleKo,titleEn,synopsis,cast,mainPageUrl,posterUrl,numOfSiteRatings
0,1,플란다스의 개,"Barking Dogs Never Bite, 2000",대학 시간강사인 고윤주(이성재)는 이번에도 교수직 추천에서 보기 좋게 떨어진다. 돈...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=1,https://img1.daumcdn.net/thumb/C408x596/?fname...,126.0
1,2,카라,"Calla, 1999",팬시회사 디자이너인 선우(송승헌)는 출근길 버스에서 지희(김희선)를 보고 첫눈에 반...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=2,https://img1.daumcdn.net/thumb/C408x596/?fname...,23.0
2,3,주노명 베이커리,2000,빵과 결혼에 관한 性스러운 이야기 빵을 만드는 기쁨이 이런 것 아닐까? 사랑이 빵을...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=3,None,15.0
3,4,여고괴담 두번째 이야기,"Memento Mori, 1999",지각을 하는 바람에 학교 담을 넘어 등교한 민아(김민선)는 수돗가에서 우연히 붉은 ...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=4,https://img1.daumcdn.net/thumb/C408x596/?fname...,74.0
4,5,행복한 장의사,2000,장판돌(오현경)은 시골의 한 마을에서 평생 장의사로 살아온 사람이다. 그의 손자 재...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=5,https://img1.daumcdn.net/thumb/C408x596/?fname...,53.0
...,...,...,...,...,...,...,...,...
30218,176047,땡큐 아임 소리,"Thank You, I'm Sorry, Tack och förlåt, 2023",출산이 멀지 않은 상황에서 갑자기 혼자가 되어버린 사라. 그런데 구원의 손길이 전혀...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=1...,https://img1.daumcdn.net/thumb/C408x596/?fname...,1.0
30219,176048,헬 캠프: 청소년 지옥 훈련소,"Hell Camp: Teen Nightmare, 2023",None,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=1...,https://img1.daumcdn.net/thumb/C408x596/?fname...,1.0
30220,176049,가상의 가상화폐,"Bitconned, 2024",None,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=1...,https://img1.daumcdn.net/thumb/C408x596/?fname...,1.0
30221,176148,쿵푸팬더4,"Kung Fu Panda 4, 2024",용의 전사 ‘포’(잭 블랙)가 상대의 기술을 그대로 카피하는 강력한 빌런 ‘카멜레온...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=1...,https://img1.daumcdn.net/thumb/C408x596/?fname...,1.0


In [45]:
daum_movies_movieid = daum_movies[['movieId']]

In [46]:
daum_movies_movieid

,movieId
0,1
1,2
2,3
3,4
4,5
...,...
30218,176047
30219,176048
30220,176049
30221,176148


In [47]:
daum_ratings

,nickName,movieId,rating,timestamp,userId,review
0,매력적그녀,121721,8,1.702394e+09,None,이런 영화 안좋아하는데 어쩔수 없이 봤다가 솔직히 기분은 진짜 별로였음 근데 또 영...
1,매력적그녀,1660,10,1.701700e+09,None,첨엔 내가 주인공이고 이렇게 술술 풀리는 행복한 인생 부럽다 이러다가 갈수록 숨막히...
2,매력적그녀,164920,10,1.692027e+09,None,처음엔 넷플에.떴길래 별 생각없이 보기 시작했는데 너무 잔잔해서 뭐지 하다가 이상하...
3,매력적그녀,130710,1,1.684073e+09,None,이거 왜 영화로 만든건가요?
4,매력적그녀,1953,6,1.677165e+09,None,당시엔 평이 엄청 낮고 여주 혹평이던 영화도 시간이 지나면 이렇게 평점이 좋아지는구...
...,...,...,...,...,...,...
582312,아림,96030,10,1.614238e+09,None,👍👍👍👍👍👍
582313,아림,134684,9,1.593245e+09,None,사람들마다 취향이 다르니 재미없으신 분들도 계시겠지만 부산행+엑시트 느낌이라 재밌었...
582314,버들치,48253,10,1.482238e+09,None,가슴이 아프다..
582315,이뿐수기,45801,9,1.516678e+09,None,추자현하면 이 영화가 생각남


,movieId,mean,count
6437,33335,10.0,4
5928,30103,10.0,1
5942,30182,10.0,1
21094,106973,10.0,1
21095,106974,10.0,1
...,...,...,...
20197,101049,0.0,1
9504,44097,0.0,1
20218,101210,0.0,1
20220,101223,0.0,1


In [ ]:
grouped_df[count]

In [95]:
daum_movies[daum_movies['num_of_collected_ratings'] > 500].sort_values('rating_mean', ascending=False).head(100)

KeyError: 'num_of_collected_ratings'

In [ ]:
pop_movies_id = \
    daum_movies[daum_movies['num_of_collected_ratings'] > 100].sort_values('rating_mean', ascending=False).head(
        100)[
        'movieId'].tolist()
# print(f"get popular movies..done")